In [3]:
import os,sys
sys.path.append(os.path.abspath('./../../'))
from Controllers.MarkedCellController import MarkedCellController,CellSources
from Controllers.SqlController import SqlController
from model.annotation_points import MarkedCellView
from sqlalchemy import inspect
inst = inspect(MarkedCellView)
attr_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
import pandas as pd
from cell_extractor.DetectorMetrics import AnnotationProximityTool
from cell_extractor.CellDetector import CellDetector
import numpy as np
from collections import Counter
user_look_up = {38:'Marissa',41:'Julian',2:'Beth'}

def get_DataFrame_from_query_result(results,category,factor):
    values = []
    for i in results:
        source = i.source.value
        if '-' in source:
            source = source.split('-')[1]    
        x,y,z = np.array([i.x,i.y,i.z]).astype(float)/factor
        values.append([x,y,z,f'{category}_{user_look_up[i.FK_annotator_id]}_{source}'])
    # values = [[eval(f'j.{i}')  for j in results ] for i in attr_names]
    df = pd.DataFrame(dict(zip(['x','y','section','name'],np.array(values).T)))
    df["x"] = pd.to_numeric(df["x"])
    df["y"] = pd.to_numeric(df["y"])
    df["section"] = pd.to_numeric(df["section"])
    return df

def get_DataFrame_from_detection_df(df,label):
    data = np.array([df.col.to_numpy().astype(int),df.row.to_numpy().astype(int),\
        df.section.to_numpy().astype(int)])
    data = pd.DataFrame(data.T,columns=['x','y','section'])
    data['name'] = [label]*len(df)
    return data

In [4]:
animal = 'DK55'
controller = SqlController()
factor = controller.get_resolution(animal)
controller = MarkedCellController()

In [5]:
search_dict = {'FK_prep_id':animal,'FK_cell_type_id':5}
qcs = controller.get_marked_cells(search_dictionary=search_dict)
positives = [i for i in qcs if i.source == CellSources.HUMAN_POSITIVE]
negatives = [i for i in qcs if i.source == CellSources.HUMAN_NEGATIVE]
positives = get_DataFrame_from_query_result(positives,'Round1',factor)
negatives = get_DataFrame_from_query_result(negatives,'Round1',factor)
qcs = get_DataFrame_from_query_result(qcs,'Round1',factor) 

In [11]:
detector = CellDetector(animal,round=2)
detections = detector.load_detections()
sures = detections[detections.predictions==2]
unsures = detections[detections.predictions==0]
sures = get_DataFrame_from_detection_df(sures,'sures')
unsures = get_DataFrame_from_detection_df(unsures,'unsures')
tool = AnnotationProximityTool()
tool.pair_distance=30
tool.set_annotations_to_compare(pd.concat([sures,unsures,qcs]))
tool.find_equivalent_points()
print(f'total sure {len(sures)} total unsure {len(unsures)}')
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 16654
after removing duplicates 15346
total sure 13067 total unsure 1726


[(('sures',), 12248),
 (('unsures',), 1256),
 (('sures', 'Round1_Beth_POSITIVE'), 677),
 (('Round1_Beth_POSITIVE',), 381),
 (('unsures', 'Round1_Beth_NEGATIVE'), 279),
 (('Round1_Beth_NEGATIVE',), 180),
 (('unsures', 'Round1_Beth_POSITIVE'), 161),
 (('sures', 'Round1_Beth_NEGATIVE'), 128),
 (('unsures', 'Round1_Beth_NEGATIVE', 'Round1_Beth_POSITIVE'), 15),
 (('unsures', 'unsures'), 4),
 (('sures', 'Round1_Beth_POSITIVE', 'Round1_Beth_POSITIVE'), 3),
 (('sures', 'Round1_Beth_NEGATIVE', 'Round1_Beth_POSITIVE'), 3),
 (('sures', 'unsures'), 3),
 (('unsures', 'Round1_Beth_POSITIVE', 'Round1_Beth_POSITIVE'), 2),
 (('Round1_Beth_NEGATIVE', 'Round1_Beth_POSITIVE'), 2),
 (('sures', 'sures'), 1),
 (('sures', 'sures', 'unsures'), 1),
 (('sures', 'Round1_Beth_NEGATIVE', 'Round1_Beth_NEGATIVE'), 1),
 (('unsures',
   'Round1_Beth_NEGATIVE',
   'Round1_Beth_POSITIVE',
   'Round1_Beth_POSITIVE'),
  1)]

In [9]:
detector = CellDetector(animal,round=1)
detections = detector.load_detections()
sures = detections[detections.predictions==2]
unsures = detections[detections.predictions==0]
sures = get_DataFrame_from_detection_df(sures,'sures')
unsures = get_DataFrame_from_detection_df(unsures,'unsures')
tool = AnnotationProximityTool()
tool.pair_distance=30
tool.set_annotations_to_compare(pd.concat([sures,unsures,qcs]))
tool.find_equivalent_points()
print(f'total sure {len(sures)} total unsure {len(unsures)}')
Counter([tuple(i) for i in tool.pair_categories.values()]).most_common()

[Errno 2] No such file or directory: '//net/birdstore/Active_Atlas_Data//cell_segmentation/models/models_from_qc_round_1_threshold_2000.pkl'
calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 15505
after removing duplicates 14064
total sure 12224 total unsure 1420


[(('sures',), 12099),
 (('unsures', 'Round1_Beth_POSITIVE'), 772),
 (('unsures', 'Round1_Beth_NEGATIVE'), 502),
 (('Round1_Beth_POSITIVE',), 422),
 (('unsures',), 127),
 (('sures', 'Round1_Beth_NEGATIVE'), 82),
 (('sures', 'Round1_Beth_POSITIVE'), 29),
 (('unsures', 'Round1_Beth_NEGATIVE', 'Round1_Beth_POSITIVE'), 12),
 (('sures', 'Round1_Beth_NEGATIVE', 'Round1_Beth_POSITIVE'), 8),
 (('sures', 'sures'), 3),
 (('unsures', 'Round1_Beth_POSITIVE', 'Round1_Beth_POSITIVE'), 3),
 (('Round1_Beth_NEGATIVE',), 2),
 (('unsures', 'unsures', 'Round1_Beth_NEGATIVE'), 1),
 (('unsures',
   'Round1_Beth_NEGATIVE',
   'Round1_Beth_POSITIVE',
   'Round1_Beth_POSITIVE'),
  1),
 (('unsures', 'Round1_Beth_NEGATIVE', 'Round1_Beth_NEGATIVE'), 1)]